In [1]:
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [ ]:
from langchain.agents import AgentState

class CustomState(AgentState): # 사용자의 선호색을 추적하는 맞춤 State
    favourite_colour: str # 앞에서의 runtime_context와 달리 기본 값을 넣지 못함

## Write to state

In [3]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

# 에이전트가 도구로 런타임에 접근하도록 함수 정의
# Command 함수: LangGraph의 특수 객체로, 그래프의 state를 직접 수정할 수 있음
# "favourite_colour": favourite_colour → CustomState.favourite_colour를 업데이트
# "messages": [ToolMessage(...)] → 메시지 히스토리에 tool 응답을 추가
# runtime.tool_call_id → LLM의 tool_call 요청에 대응하는 ID. ToolMessage에 필수
@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    "gpt-5-nano",
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [6]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="내가 좋아하는 색은 녹색이야.")]},
    {"configurable": {"thread_id": "1"}}
)

In [7]:
from pprint import pprint

pprint(response)

{'favourite_colour': '녹색',
 'messages': [HumanMessage(content='내가 좋아하는 색은 녹색이야.', additional_kwargs={}, response_metadata={}, id='4ac0693a-8852-4112-897f-af6c6474ce99'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 349, 'prompt_tokens': 148, 'total_tokens': 497, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D6dXlSG2vjQcnlpThIBlaGzx0vmL2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c387e-8ecd-72e1-8d1e-1da2799e5a1b-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': '녹색'}, 'id': 'call_Y8xHtuvbbUbgxRvS6y6AGoVP', 'type': 'tool_call'}], invalid_tool_calls=[], usa

In [ ]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="안녕하세요?")],
        "favourite_colour": "green" # state 초기값 직접 설정
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='7d534cd6-7698-4492-98e2-e4e9017d4e9c'),
              AIMessage(content='안녕하세요! 무엇을 도와드릴까요? 정보 검색, 번역, 코딩 도움, 공부 보조, 일상 대화 연습 등 다양한 주제로 함께할 수 있어요. 원하는 것이 있으면 말씀해 주세요.\n\n참고로 원하신다면 선호 색상 같은 개인 설정을 저장해 드릴 수도 있습니다. 지금 시작하고 싶은 주제가 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 665, 'prompt_tokens': 139, 'total_tokens': 804, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D6dYfHvgXyoI9545arBNXYKC7Pyii', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c387f-6153-70b2-8c9d-3ec4245623e8-0', tool_calls=[], invalid_tool

## Read state

In [9]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    "gpt-5-nano",
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [10]:
response = agent.invoke(
    { "messages": [HumanMessage(content="내가 좋아하는 색은 녹색이야.")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': '녹색',
 'messages': [HumanMessage(content='내가 좋아하는 색은 녹색이야.', additional_kwargs={}, response_metadata={}, id='0e1a7d90-12cf-4ae9-b004-72bc347a4d6c'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 477, 'prompt_tokens': 169, 'total_tokens': 646, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D6dgqrhhgjtMT1vY3kIEjCAVyrkOC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3887-288c-7021-a76f-53c248961c82-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': '녹색'}, 'id': 'call_4iYhKOyOUmT59FNVZABQ2sGN', 'type': 'tool_call'}], invalid_tool_calls=[], usa

In [11]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': '녹색',
 'messages': [HumanMessage(content='내가 좋아하는 색은 녹색이야.', additional_kwargs={}, response_metadata={}, id='0e1a7d90-12cf-4ae9-b004-72bc347a4d6c'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 477, 'prompt_tokens': 169, 'total_tokens': 646, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D6dgqrhhgjtMT1vY3kIEjCAVyrkOC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3887-288c-7021-a76f-53c248961c82-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': '녹색'}, 'id': 'call_4iYhKOyOUmT59FNVZABQ2sGN', 'type': 'tool_call'}], invalid_tool_calls=[], usa